BERT Model Implementation for Classification (Without Strides max length 512 words)

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Input.csv')

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'].values, df['Label'].values, test_size=0.2, random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
class LegalCaseDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LegalCaseDataset(train_encodings, train_labels)
val_dataset = LegalCaseDataset(val_encodings, val_labels)

In [4]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='epoch',
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
trainer.train()

  7%|▋         | 10/153 [01:56<26:49, 11.26s/it]

{'loss': 1.4151, 'grad_norm': 18.000797271728516, 'learning_rate': 4.673202614379085e-05, 'epoch': 0.2}


 13%|█▎        | 20/153 [03:43<23:46, 10.73s/it]

{'loss': 1.2746, 'grad_norm': 9.958771705627441, 'learning_rate': 4.3464052287581704e-05, 'epoch': 0.39}


 20%|█▉        | 30/153 [05:36<22:37, 11.04s/it]

...

                                                 
100%|██████████| 153/153 [31:53<00:00, 10.29s/it]

{'eval_loss': 0.36355268955230713, 'eval_runtime': 42.5214, 'eval_samples_per_second': 2.399, 'eval_steps_per_second': 0.306, 'epoch': 3.0}


100%|██████████| 153/153 [31:55<00:00, 12.52s/it]

{'train_runtime': 1915.9096, 'train_samples_per_second': 0.634, 'train_steps_per_second': 0.08, 'train_loss': 0.6323285546957278, 'epoch': 3.0}


TrainOutput(global_step=153, training_loss=0.6323285546957278, metrics={'train_runtime': 1915.9096, 'train_samples_per_second': 0.634, 'train_steps_per_second': 0.08, 'total_flos': 319685672816640.0, 'train_loss': 0.6323285546957278, 'epoch': 3.0})

In [6]:
model.save_pretrained('./fine-tuned-bert512')
tokenizer.save_pretrained('./fine-tuned-bert512')
def classify_case(text):
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**encoding)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class_id])[0]

example_text = '''A property owner files a lawsuit against their neighbor for encroaching on their land and demands that the boundary be restored to its rightful place.'''
prediction = classify_case(example_text)
print(f"Predicted Case Type: {prediction}")

Predicted Case Type: Civil Case


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
tokenizer = BertTokenizer.from_pretrained('./fine-tuned-bert512')
model = BertForSequenceClassification.from_pretrained('./fine-tuned-bert512')
label_encoder = joblib.load('label_encoder.joblib')

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def classify_case(text):
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**encoding)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class_id])[0]

example_text = '''A property owner files a lawsuit against their neighbor for encroaching on their land and demands that the boundary be restored to its rightful place.'''
prediction = classify_case(example_text)
print(f"Predicted Case Type: {prediction}")

Predicted Case Type: Civil Case
